In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy import sparse
import numpy as np

In [2]:
dance_ratings = pd.read_csv('dance_ratings.csv')
kdance = pd.read_csv('kdance.csv', encoding='latin-1')
kd_ratings = pd.merge(kdance,dance_ratings)

In [3]:
kd_ratings['song'] = kd_ratings['title'] +' - '+ kd_ratings['artist']
print(kd_ratings.shape)
kd_ratings.head()

(1776, 7)


,song_id,title,artist,tempo,user_id,rating,song
0,12,OOMM (Out of My Mind),3YE,150.063,6,1,OOMM (Out of My Mind) - 3YE
1,12,OOMM (Out of My Mind),3YE,150.063,9,2,OOMM (Out of My Mind) - 3YE
2,12,OOMM (Out of My Mind),3YE,150.063,24,3,OOMM (Out of My Mind) - 3YE
3,12,OOMM (Out of My Mind),3YE,150.063,60,3,OOMM (Out of My Mind) - 3YE
4,12,OOMM (Out of My Mind),3YE,150.063,68,2,OOMM (Out of My Mind) - 3YE


In [4]:
numdance_rating = kd_ratings.groupby('song')['rating'].count().reset_index()

In [5]:
numdance_rating.head()

,song,rating
0,A-TEEN - SEVENTEEN,12
1,After School - Weeekly,17
2,All Night - ASTRO,17
3,Always You - ASTRO,21
4,BBIBBI - IU,16


In [6]:
numdance_rating.rename(columns={'rating':'no of ratings'}, inplace=True)
final_dance = kd_ratings.merge(numdance_rating, on='song')
final_dance

,song_id,title,artist,tempo,user_id,rating,song,no of ratings
0,12,OOMM (Out of My Mind),3YE,150.063,6,1,OOMM (Out of My Mind) - 3YE,17
1,12,OOMM (Out of My Mind),3YE,150.063,9,2,OOMM (Out of My Mind) - 3YE,17
2,12,OOMM (Out of My Mind),3YE,150.063,24,3,OOMM (Out of My Mind) - 3YE,17
3,12,OOMM (Out of My Mind),3YE,150.063,60,3,OOMM (Out of My Mind) - 3YE,17
4,12,OOMM (Out of My Mind),3YE,150.063,68,2,OOMM (Out of My Mind) - 3YE,17
...,...,...,...,...,...,...,...,...
1771,622,Hold,WINNER,155.930,105,0,Hold - WINNER,7
1772,622,Hold,WINNER,155.930,116,0,Hold - WINNER,7
1773,622,Hold,WINNER,155.930,209,2,Hold - WINNER,7
1774,622,Hold,WINNER,155.930,327,5,Hold - WINNER,7


In [7]:
final_dance = final_dance[final_dance['no of ratings']>=5] #only songs above 5 users have rated is qualified, to avoid noise
final_dance.shape

(1776, 8)

In [8]:
final_dance.drop_duplicates(['user_id','song'], inplace=True)
final_dance.shape

(1744, 8)

In [9]:
dance_pivot = final_dance.pivot_table(columns='user_id', index='song', values='rating')
dance_pivot.fillna(0, inplace=True)

In [10]:
print(dance_pivot.shape)
dance_pivot

(111, 329)


user_id,1,2,3,4,5,6,7,8,9,10,...,323,324,325,326,327,328,329,330,331,332
song,,,,,,,,,,,,,,,,,,,,,
A-TEEN - SEVENTEEN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
After School - Weeekly,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
All Night - ASTRO,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0
Always You - ASTRO,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BBIBBI - IU,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WINE (Feat.Changmo) (Prod. SUGA) - SURAN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
What You Wanted - CIX,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
What is Love - TWICE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
from scipy.sparse import csr_matrix
dance_sparse = csr_matrix(dance_pivot)
type(dance_sparse)

scipy.sparse.csr.csr_matrix

In [12]:
from sklearn.neighbors import NearestNeighbors
dance_model = NearestNeighbors(algorithm='brute')

In [13]:
dance_model.fit(dance_sparse)

NearestNeighbors(algorithm='brute')

In [14]:
distances, suggestions = dance_model.kneighbors(dance_pivot.iloc[4,:].values.reshape(1,-1), n_neighbors=15)

In [15]:
suggestions

array([[  4,  86,  38,  26,  53,  23,  17, 108,   0, 101,  52, 105,  42,
        103,   9]], dtype=int64)

In [16]:
for i in range(len(suggestions)):
    print(dance_pivot.index[suggestions[i]])

Index(['BBIBBI - IU', 'Really - BLACKPINK', 'Hold - WINNER',
       'HALA HALA (Hearts Awakened, Live Alive) - ATEEZ',
       'L.O.V.E - PARK JI HOON', 'Eung Eung - Apink', 'Dionysus - BTS',
       'What is Love - TWICE', 'A-TEEN - SEVENTEEN', 'TWILIGHT - WEi',
       'Ko Ko Bop - EXO', 'WHERE YOU AT - NU'EST W', 'I Need Somebody - DAY6',
       'Taller than You - MAMAMOO', 'Black Swan - BTS'],
      dtype='object', name='song')


In [17]:
dance_pivot.index[50]

'JUICY - Rocket Punch'

In [18]:
np.where(dance_pivot.index=='JUICY - Rocket Punch')[0][0]

50

In [25]:
def recommend_kmusic_dance(song):
    song_id=np.where(dance_pivot.index==song)[0][0]
    distances, suggestions = dance_model.kneighbors(dance_pivot.iloc[song_id,:].values.reshape(1,-1), n_neighbors=15)
    
    for i in range(len(suggestions)):
        print(dance_pivot.index[suggestions[i]])

In [26]:
recommend_kmusic_dance('JUICY - Rocket Punch')

Index(['JUICY - Rocket Punch', 'Hold - WINNER', 'Really - BLACKPINK',
       'Dionysus - BTS', 'L.O.V.E - PARK JI HOON',
       'HALA HALA (Hearts Awakened, Live Alive) - ATEEZ', 'Eung Eung - Apink',
       'What is Love - TWICE', 'TWILIGHT - WEi', 'I Need Somebody - DAY6',
       'Taller than You - MAMAMOO', 'WHERE YOU AT - NU'EST W',
       'Mikrokosmos - BTS', 'Black Swan - BTS', 'Ice Cream Cake - Red Velvet'],
      dtype='object', name='song')


In [27]:
ratedance = kd_ratings.groupby('song_id')['rating'].count().reset_index()
ratedance

,song_id,rating
0,12,17
1,13,20
2,14,15
3,25,14
4,30,19
...,...,...
106,612,17
107,613,17
108,614,6
109,619,15


In [28]:
dance_list = pd.merge(kdance,ratedance)
dance_list.rename(columns={'rating':'no of ratings'}, inplace=True)
dance_list 

,song_id,title,artist,tempo,no of ratings
0,12,OOMM (Out of My Mind),3YE,150.063,17
1,13,Crazy,4Minute,176.019,20
2,14,Hate,4Minute,149.883,15
3,25,Savage,aespa,146.959,14
4,30,Daechwita,Agust D,144.995,19
...,...,...,...,...,...
106,612,After School,Weeekly,164.007,17
107,613,Tag Me,Weeekly,160.007,17
108,614,TWILIGHT,WEi,156.110,6
109,619,Picky Picky,Weki Meki,150.001,15


In [29]:
dance_list['song'] = dance_list['title'] +' - '+ dance_list['artist']
dance_list 

,song_id,title,artist,tempo,no of ratings,song
0,12,OOMM (Out of My Mind),3YE,150.063,17,OOMM (Out of My Mind) - 3YE
1,13,Crazy,4Minute,176.019,20,Crazy - 4Minute
2,14,Hate,4Minute,149.883,15,Hate - 4Minute
3,25,Savage,aespa,146.959,14,Savage - aespa
4,30,Daechwita,Agust D,144.995,19,Daechwita - Agust D
...,...,...,...,...,...,...
106,612,After School,Weeekly,164.007,17,After School - Weeekly
107,613,Tag Me,Weeekly,160.007,17,Tag Me - Weeekly
108,614,TWILIGHT,WEi,156.110,6,TWILIGHT - WEi
109,619,Picky Picky,Weki Meki,150.001,15,Picky Picky - Weki Meki


In [30]:
import pickle

In [31]:
pickle.dump(dance_list,open('kdance.pkl','wb'))

In [32]:
dance_list['song']

0      OOMM (Out of My Mind) - 3YE
1                  Crazy - 4Minute
2                   Hate - 4Minute
3                   Savage - aespa
4              Daechwita - Agust D
                  ...             
106         After School - Weeekly
107               Tag Me - Weeekly
108                 TWILIGHT - WEi
109        Picky Picky - Weki Meki
110                  Hold - WINNER
Name: song, Length: 111, dtype: object

In [37]:
dance_list.to_dict()

{'song_id': {0: 12,
  1: 13,
  2: 14,
  3: 25,
  4: 30,
  5: 38,
  6: 48,
  7: 49,
  8: 50,
  9: 55,
  10: 57,
  11: 58,
  12: 59,
  13: 60,
  14: 85,
  15: 86,
  16: 100,
  17: 103,
  18: 106,
  19: 107,
  20: 111,
  21: 112,
  22: 125,
  23: 127,
  24: 133,
  25: 139,
  26: 140,
  27: 143,
  28: 144,
  29: 151,
  30: 159,
  31: 169,
  32: 170,
  33: 171,
  34: 173,
  35: 174,
  36: 175,
  37: 177,
  38: 182,
  39: 183,
  40: 186,
  41: 188,
  42: 189,
  43: 195,
  44: 202,
  45: 210,
  46: 214,
  47: 216,
  48: 232,
  49: 242,
  50: 257,
  51: 266,
  52: 268,
  53: 273,
  54: 277,
  55: 294,
  56: 316,
  57: 325,
  58: 326,
  59: 345,
  60: 351,
  61: 371,
  62: 375,
  63: 383,
  64: 386,
  65: 391,
  66: 392,
  67: 397,
  68: 399,
  69: 405,
  70: 420,
  71: 423,
  72: 427,
  73: 428,
  74: 430,
  75: 436,
  76: 437,
  77: 438,
  78: 445,
  79: 447,
  80: 448,
  81: 451,
  82: 456,
  83: 467,
  84: 468,
  85: 469,
  86: 471,
  87: 480,
  88: 485,
  89: 493,
  90: 503,
  91: 515,
  9

In [33]:
pickle.dump(dance_list.to_dict(),open('kdance_dict.pkl','wb'))

In [34]:
pickle.dump(dance_pivot,open('dance_pivot.pkl','wb'))

In [35]:
pickle.dump(dance_model,open('dance_model.pkl','wb'))

In [36]:
dance_list['tempo']

0      150.063
1      176.019
2      149.883
3      146.959
4      144.995
        ...   
106    164.007
107    160.007
108    156.110
109    150.001
110    155.930
Name: tempo, Length: 111, dtype: float64